# DC Language Immersion: Mapping Addresses to Feeder Schools 

<img src="https://i0.wp.com/dcimmersion.org/wp-content/uploads/2016/04/cropped-dcimmersion_linkedin_logo_b.png?fit=512%2C512" style="float:right; height: 50px; width: 50px; padding: 0em 1em 0em 1em;"></img>



---
**Author** Glenn Abastillas | **Created** April, 14, 2019 | **Category** Volunteer Work | Language Not-for-Profit

This notebook shows how to assign addresses to school districts in an automated way. The address information comes from the DC Language Immersion Google Drive storage. The school district and ward data comes from the [DC Government's Open Data](www.opendata.dc.gov) APIs.

Update:
    1. July 10, 2019: received data from DC Language Immersion to test boundary school and ward assignment.

---
#### Import Packages

In [1]:
import pandas as pd
import geopandas as gpd
import shapely as shp
import shapefile
import usaddress
import re

from geopy.geocoders import Nominatim
from random import randint

%matplotlib inline

#### Load Map Data

Define the path and label variables for loading map data without the API.

In [2]:
folder = ".."
prefix = 'School_Attendance_Zones'
levels = ['Elementary', 'Middle_School', 'Senior_High', 'Ward']

Load Data and Shape files.

In [3]:
data = {level: pd.read_csv(f"{folder}/{prefix}_{level}.csv") for level in levels}

In [4]:
shapes = {level: shapefile.Reader(f"{folder}/{prefix}_{level}/{prefix}_{level}.shp") for level in levels}

Shape methods include `bbox`, `parts`, `points`, `shapeType`, `shapeTypeName`.

Coordinate information is contained in the `points` attribute. The following cell prints out a random district shape every time it is run.

In [5]:
# Set some parameters
level = 'Elementary'
LOW, HI = 0, len(shapes[level].shapes())

# Sample a shape
random_integer = randint(LOW, HI)
example_shape = shp.geometry.Polygon(shapes[level].shape(random_integer).points)
print(f"School boundary #{random_integer}")
example_shape

Shell is not a LinearRing


School boundary #41


#### Instantiate Geocoders to Convert Addresses

Instantiate the Geocoder and use it to convert addresses into `Point` objects to allow for inclusion checking.

In [6]:
geocoder = Nominatim(user_agent="default")

Load contact data and information.

In [ ]:
raw_contacts = pd.read_csv("dc_language_immersion_contacts.csv").drop_duplicates()
# raw_contacts.columns

# Check if EUID and LEID are actually unique identifiers
print("EUID and LEID truly unique identifiers: ",
(raw_contacts.EUID.unique().shape[0] == raw_contacts.LEID.unique().shape[0] == raw_contacts.shape[0]))

In [ ]:
raw_contacts.TYPE = raw_contacts.TYPE.apply(lambda x: '; '.join([__.title() for __ in x.split(';') if __.strip()]))

contacts = raw_contacts[raw_contacts['Street Address 1'].notnull()]
contacts = contacts[['EUID', 'LEID', 'Street Address 1', 'City', 'State', 'Zip Code']]
contacts.columns = ['euid', 'leid', 'address', 'city', 'state', 'zip']
contacts.shape

### Clean Up Data
Some contacts' addresses have a DC street address, but out of state city and state designation. These functions help analyze the street address.

In [ ]:
def n_tokens(addr):
    return(len(addr.split()))

def valid_quadrant(addr):
    suffix = addr.split()[-1].upper()
    if(suffix not in ['NW', 'NE', 'SE', 'SW']):
        return(False)
    return(True)

The first time the cell below is run, it will show the invalid addresses. After running the subsequent two cells, the next cell should produce an empty `DataFrame`.

In [ ]:
contacts[contacts.state.str.lower().str.find('district') > -1]

In [ ]:
valid_quadrant_ = contacts.address.apply(valid_quadrant)
not_washington_ = ~contacts.city.apply(lambda x: 'ashi' in str(x).lower())
state_is_in_dc_ = ((contacts.state.str.lower() == 'dc') | ((contacts.state.str.lower().str.find('district') > -1)))
state_is_an_nan = contacts.state.isnull()

contacts[valid_quadrant_ & not_washington_ & (state_is_in_dc_ | state_is_an_nan)]

Replace `NaN` city names with Washington if the address is in Washington, DC. Assume that street addresses with quadrant information are in DC.

In [ ]:
# contacts.loc[contacts[valid_quadrant_ & not_washington_ & (state_is_in_dc_ | state_is_an_nan)], 'city'] = 'Washington'
# contacts.loc[contacts[valid_quadrant_ & not_washington_ & (state_is_in_dc_ | state_is_an_nan)], 'state'] = 'DC'
contacts.loc[valid_quadrant_ & not_washington_ & (state_is_in_dc_ | state_is_an_nan), 'city'] = 'Washington'
contacts.loc[valid_quadrant_ & (state_is_an_nan), 'state'] = 'DC'

Normalize the city and state data for addresses in Washington, DC. Re-running the cells above should return an empty data frame if successfully processed.

In [ ]:
contacts.loc[state_is_in_dc_, 'city'] = 'Washington'
contacts.loc[state_is_in_dc_, 'state'] = 'DC'

Save this second dataset, which contains rows of contacts with a `state=='dc'`

In [ ]:
# df2 = contacts.copy()
# df2.to_csv("contacts_washington_dc.csv", index=False)

In [ ]:
contacts[state_is_in_dc_].city.unique()

#### Address Analysis

Discover different address types for regular expression normalization. All entries had a `state='DC'`.

In [ ]:
contacts.address.unique()[:5]

In [ ]:
dc = contacts[state_is_in_dc_]
dc.shape

#### ZIP
Examine the ZIP codes for all DC rows.

In [ ]:
dc.zip.unique()

#### City
Examine the city names for all DC rows.

In [ ]:
washington = ('Washington', re.compile(r"\w+?shi\w{0,2}t\w+?|DC|Wa", re.IGNORECASE))
dc[dc.city.apply(lambda x: len(washington[1].findall(str(x))) > 0)].city.unique()

Define a function to get just the `StreetPostType` data from the street addresses.

In [ ]:
def get_address_type(street_address):
    try:
        return(usaddress.tag(street_address)[-1])
    except:
        return(None)
    

In [ ]:
dc.loc[:,'address_type'] = dc.address.apply(get_address_type)
dc.address_type.value_counts()

##### Quadrant

Check if the street address ends in a quadrant. Separate addresses into two variables.

In [ ]:
dc_valid_quadrant = dc[dc.address.apply(valid_quadrant) & dc.city.isin([washington[0]])]
dc_valid_quadrant.shape

In [ ]:
dc_invalid_quadrant = dc[~dc.address.apply(valid_quadrant) & dc.city.isin([washington[0]])]
dc_invalid_quadrant.shape

Some examples of tokens that end a street address.

In [ ]:
dc_invalid_quadrant[dc_invalid_quadrant.address.apply(n_tokens) > 1].address.apply(lambda x: x.split()[-2]).sort_values().unique()

The next cell investigates the type of suffixes for the valid quadrant data for regular expression development.

In [ ]:
dc_valid_quadrant[dc_valid_quadrant.address.apply(n_tokens) > 1].address.apply(lambda x: x.split()[-2]).sort_values().unique()

### Regular Expressions for Normalizing Addresses
Regular expressions can be used to standardize the address names. First create a shorthand variable name for the two types of quadrant classifications.

In [ ]:
dcv = dc_valid_quadrant
dci = dc_invalid_quadrant

Now create the regular expressions to detect the street entities and the replacement term. These post types were determined by examining the data.

In [ ]:
import re

types = [('alley',          r'all?e?y'), 
         ('avenue',         r'av(\w+)?\S?'), 
         ('boulevard',     r'b\w{0,2}l\w?v\w?r?d'),
         ('branch',         r'branch\S?'), 
         ('court',          r'c[ou]{0,2}r?t\S?'), 
         ('drive',          r'dr(ive)?\S?'), 
         ('lane',           r'l\w?n\w?\S?'), 
         ('place',          r'pl(ace)?\S?'), 
         ('road',           r'r[oa]{0,2}d\S?'), 
         ('street',         r'st(ree?t)?\S?'), 
         ('way',            r'wa?y')]

types_dict = dict([(a, re.compile(b, re.IGNORECASE)) for a, b in types])

Test the regular expressions on an example.

In [ ]:
# example = ' 2342 Nebraska av.'
# [types_dict[key].sub(key, example) for key in types_dict.keys()]

Define functions to normalize street address strings.

In [ ]:
def get_address_type(A):
    """ See if input address `A` is a Street Address, PO Box, Ambiguous, or None """
    try:
        return(usaddress.tag(A)[-1])
    except:
        return(None)

def get_occupancy_identifier(address):
    if(address):
        try:
            parsed, street_type = usaddress.tag(address)
            return(parsed['OccupancyIdentifier'])
        except:
            return(None)
    return(address)
    
def normalize_posttype(P, tdict=types_dict):
    for key in tdict:
        regex = tdict[key]
        if(regex.findall(P)):
            return(regex.sub(key, P))
    return(P)

def normalize_address(A):
    try:
        parsed, street_type = usaddress.tag(A)
        A = parsed['AddressNumber'].replace('.', '')
        B = parsed['StreetName'].replace('.', '')
        
        # Process StreetNames if numeric
        B = B.lower() if B[0].isdigit() else B.title()
        B = f"{B}th" if (B.isdigit()) else (f"{B[:-2]}th" if (B[:-2].isdigit()) else B)
        B = f"{B[:-2]}st" if B[:-2].isdigit() and (B[:-2] == '1' or B[-3] == '1') else B
        B = f"{B[:-2]}nd" if B[:-2].isdigit() and (B[:-2] == '2' or B[-3] == '2' and B[0] != '1') else B
        B = f"{B[:-2]}rd" if B[:-2].isdigit() and (B[:-2] == '3' or B[-3] == '3' and B[0] != '1') else B
        
        C = normalize_posttype(parsed['StreetNamePostType']).title()
            
        try:
            D = parsed['StreetNamePostDirectional'].upper()
        except:
            D = ''
        return(f"{A} {B} {C} {D}".replace('.', '').strip())
        
    except:
#         print(A)
        return(None)

Normalize the addresses of the original dataset `contacts`.

In [ ]:
contacts = contacts.assign(address_type=contacts.address.apply(get_address_type),
                           occupancy_identifier=contacts.address.apply(get_occupancy_identifier))
contacts.address_type.value_counts()

In [ ]:
contacts.loc[contacts.address_type.isin(['Street Address']),].assign(address_norm=contacts.address.apply(normalize_address)).sample(10)

In [ ]:
dc = contacts[state_is_in_dc_ & (contacts.address_type == 'Street Address')]
not_dc = contacts[~state_is_in_dc_ & (contacts.address_type == 'Street Address')]
print(f"R,C: {dc.shape[0]} contacts with Street Addresses in DC\n"
      f"R,C: {not_dc.shape[0]} contacts without Street Addresses in DC\n"
      f"R,C: {contacts.shape[0]} total contacts.")

### Assign Dataset to `current`

Assign which dataset to focus on with the `current` variable.

In [ ]:
current = dc

Continue to process the street addresses.

In [ ]:
current_st = current[current.address_type.isin(['Street Address'])]

print(current_st.shape)
current_st.address.sample(10).apply(normalize_address)

In [ ]:
current_st = current_st.assign(address_clean=current_st.address.apply(normalize_address))

In [ ]:
# address = "1160 1st Street NE, Washington DC 20002"
address = ', '.join(current_st.sample()[['address_clean', 'city', 'state', 'zip']].iloc[0])
location = geocoder.geocode(address)
print(address)

Get the location point coordinates and check for inclusion.

In [ ]:
location_point = shp.geometry.Point(tuple(location.point))
example_shape.contains(location_point)

#### Tester to Find School District for Address

First design the logic then create the function definition.

In [ ]:
level = levels[2]
SIZE = 0, len(shapes[level].shapes())

for i in range(len(SIZE)):
    current = shp.geometry.Polygon(shapes[level].shape(i).points)
    if(current.contains(location_point)):
        break

# shape = shapes[level].shape(i)        
print(f"Your {level} school for {address} is {i}.")
current

In [ ]:
location

In [ ]:
from shapely.geometry import Point, Polygon
from geopy.geocoders import Here
from geopy.exec import GeocoderTimedOut

geocoder = Here(app_id="Yc3DwCnHtBBIqRwBCyHS",
                app_code="M_6pX8o174Ny1Ifs536-kg")

def get_coordinates(address, i=3):
    """ Return the longitude, latitude, and altitude of an address as a tuple """
    if(i > -1)
    if(address):
        try:
            return(tuple(geocoder.geocode(address).point))
        except(GeocoderTimedOut):
            return(get_coordinates(address, i-1))
    return(None)
    
def assign_school_zone(address, school_data=data, shape_data=shapes, school_levels=levels, level=0):
    """ Return school name, object ID, and Ward number. """
    if(address):
        longitude, latitude, altitude = address

        point = Point((latitude, longitude))
        school = school_levels[level] if isinstance(level, int) else level

        for i, shape in enumerate(shape_data[school].shapes()):
            polygon = Polygon(shape.points)
            if(polygon.contains(point)):
                return(i+1, f"{school_data[school].loc[i,'NAME']} {school.replace('_', ' ')}")
    return(None)


### Deploy School Zone Assignment Algorithm

Test the algorithm on a single address and then multiple addresses.

In [ ]:
coords = get_coordinates(address)
print(address, assign_school_zone(coords, data, shapes, levels, 'Middle_School'))

### Get Boundary and Ward Information

Run the cell to get `boundary school` and `ward` information for each address.

In [ ]:
from tqdm import tqdm
import time
# addresses = ['1150 1st street ne', '1822 1st St NW, Washington, DC 20001', '1918 14th St SE, Washington, DC 20020']

DATA = current_st
decide = lambda x: x if x else ''
addresses = [(f"{decide(a)} {decide(b)} {decide(c)} {decide(d)}".strip(), e, f) 
             for i, (a, b, c, d, e, f) in list(DATA[['address_clean', 'city', 'state', 'zip', 'euid', 'leid']].iterrows())]

address_boundary = list()

for address, euid, leid in tqdm(addresses):
    boundary = list()
    address_ = get_coordinates(address)

    for level in levels:
        try:
            school = assign_school_zone(address, level=level)[1]
            boundary.append(school)
        except:
            pass
    address_boundary.append(tuple([euid, leid, address] + boundary))
    time.sleep(1)

len(address_boundary)

In [ ]:
address_boundary[:3]

Address and boundary schools from the `dci` (`dc_invalid_quadrant`) data set.

In [ ]:
# ab_dci = address_boundary # Comment out once assigned to allow the list above to get reassigned
len(ab_dci)

Address and boundary schools from the `dcv` (`dc_valid_quadrant`) data set.

In [ ]:
# ab_dcv = address_boundary # Comment out once assigned to allow the list above to get reassigned
len(ab_dcv)

Create a new DataFrame to hold processed data.

In [ ]:
processed = pd.DataFrame(address_boundary, columns=['euid', 'leid', 'address', 'elementary_school', 'middle_school', 'senior_school', 'ward'])
processed.shape

In [ ]:
processed.head()

Number of addresses where there is no boundary school information available at all for the addresses.

In [ ]:
has_all_school_levels = ~(processed.senior_school.isnull() & processed.middle_school.isnull() & processed.elementary_school.isnull())

In [ ]:
processed[~has_all_school_levels].shape

Look into the address where there is a school available.

In [ ]:
processed.sample(20)

Save current processed DataFrame.

In [ ]:
processed.to_csv("./processed.csv", index=False)

#### Merge Back to Original Data

Using the street address, city, and state columns, merged cleaned and assigned data back to main dataset. First, separate out the street address.

In [ ]:
processed = processed.assign(street_address=processed.address.apply(lambda x: ' '.join(x.split()[:-3])))
processed.head()

Now, attempt to merge the datasets.

In [ ]:
merged = contacts.merge(processed, on=['euid', 'leid'])
merged.head()

In [ ]:
merged.shape

In [ ]:
merged.to_csv("./merged_1.csv", index=False)

#### Connecting via API

Sandbox for reading in School Attendance Zones for Senior High School from the Open Data repository.

In [ ]:
api_data = gpd.read_file("https://opendata.arcgis.com/datasets/d2e0486155144b6d8e11e12469a64b1c_15.geojson")
api_data.head(2)

In [ ]:
geometry = api_data['geometry']

In [ ]:
geometry.head()

In [ ]:
shape1 = Polygon(geometry[1])

In [ ]:
api_data.plot(label='NAME')

In [ ]:
geojson_copy = "https://opendata.arcgis.com/datasets/d2e0486155144b6d8e11e12469a64b1c_15.geojson"

---

# END